This exercise is adapted from https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/


## Keras Tutorial Overview

There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future.

The steps you are going to cover in this tutorial are as follows:

1. Load Data.
2. Define Keras Model.
3. Compile Keras Model.
4. Fit Keras Model.
5. Evaluate Keras Model.
6. Tie It All Together.
7. Make Predictions

This Keras tutorial has a few requirements:

1. You have Python 2 or 3 installed and configured.
2. You have SciPy (including NumPy) installed and configured.
3. You have Keras and a backend (Theano or TensorFlow) installed and configured.

### 1. Load Data
The first step is to define the functions and classes we intend to use in this tutorial.

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

The imports required are listed below.

In [1]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


Now load the data from *file pima-indians-diabetes.data.csv* stored in Data folder (check Google Drive if unsure)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# load the dataset
df_dataset = pd.read_csv('pima-indians-diabetes.data.csv',header=None)

# split into input (X) and output (y) variables
X = df_dataset.values[:, :8]
y = df_dataset.values[:,8]


In [6]:
df_dataset.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**Q:** Find out more about the dataset - such as how many columns, what kind of information does it provide etc.

In [9]:
df_dataset.describe()

,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### 2. Define Keras Model
Models in Keras are defined as a sequence of layers.

We create a **Sequential model** and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the **input_dim** argument and setting it to 8 for the 8 input variables.

*Question:* How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

We will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

* The model expects rows of data with 8 variables (the input_dim=8 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has one node and uses the sigmoid activation function.

In [26]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(1, input_dim=8, activation='sigmoid'))

### 3. Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

We must specify the loss function to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the **loss** argument. This loss is for a binary classification problems and is defined in Keras as **“binary_crossentropy“**. 

We will define the optimizer as the efficient stochastic gradient descent algorithm “adam“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. 

Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the metrics argument.

In [28]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

* **Epoch:** One pass through all of the rows in the training dataset.
* **Batch:** One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [12]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 0s 684us/step - loss: 9.7594 - accuracy: 0.3581
Epoch 2/150
77/77 [==============================] - 0s 684us/step - loss: 8.8138 - accuracy: 0.4167
Epoch 3/150
77/77 [==============================] - 0s 712us/step - loss: 8.2509 - accuracy: 0.4583
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 7.7993 - accuracy: 0.4857
Epoch 5/150
77/77 [==============================] - 0s 770us/step - loss: 7.4602 - accuracy: 0.5104
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 6.5709 - accuracy: 0.5625
Epoch 7/150
77/77 [==============================] - 0s 935us/step - loss: 7.1543 - accuracy: 0.5260
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 7.5944 - accuracy: 0.5039
Epoch 9/150
77/77 [==============================] - 0s 985us/step - loss: 7.5751 - accuracy: 0.5052
Epoch 10/150
77/77 [==============================] - 0s 818us/step - loss: 7.3639 - accuracy: 0.

**Q:** change the value of Epoch - what differences will you expect/see?

In [29]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 680us/step - loss: 8.0693 - accuracy: 0.6484
Epoch 2/300
26/26 [==============================] - 0s 680us/step - loss: 2.2518 - accuracy: 0.6133
Epoch 3/300
26/26 [==============================] - 0s 680us/step - loss: 1.7296 - accuracy: 0.5938
Epoch 4/300
26/26 [==============================] - 0s 800us/step - loss: 1.4152 - accuracy: 0.5651
Epoch 5/300
26/26 [==============================] - 0s 744us/step - loss: 1.2924 - accuracy: 0.5417
Epoch 6/300
26/26 [==============================] - 0s 759us/step - loss: 1.1890 - accuracy: 0.5208
Epoch 7/300
26/26 [==============================] - 0s 822us/step - loss: 1.0974 - accuracy: 0.5078
Epoch 8/300
26/26 [==============================] - 0s 760us/step - loss: 1.0353 - accuracy: 0.5000
Epoch 9/300
26/26 [==============================] - 0s 736us/step - loss: 0.9777 - accuracy: 0.5078
Epoch 10/300
26/26 [==============================] - 0s 741us/step - loss: 0.9356 - accura

### 5. Evaluate Keras Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the **evaluate()** function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The **evaluate()** function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [30]:
# evaluate the keras model
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 609us/step - loss: 0.5434 - accuracy: 0.7031
Accuracy: 70.31


### 7. Make Predictions
We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [31]:
# make probability predictions with the model
# evaluate the keras model
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]



**Q:** now redo predictions by using the function predict_classes(). Show the outcome of the first 10 cases (code provided)

In [16]:
predictions = model.predict_classes(X)

for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

AttributeError: 'Sequential' object has no attribute 'predict_classes'

## Challenge 1:
Separate the training and test datasets (70% vs 30%) and rerun the evaluation

In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model.fit(x_train, y_train, epochs=300, batch_size=30)
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/300
18/18 [==============================] - 0s 824us/step - loss: 0.5709 - accuracy: 0.6909
Epoch 2/300
18/18 [==============================] - 0s 765us/step - loss: 0.5475 - accuracy: 0.7095
Epoch 3/300
18/18 [==============================] - 0s 765us/step - loss: 0.5440 - accuracy: 0.7169
Epoch 4/300
18/18 [==============================] - 0s 901us/step - loss: 0.5371 - accuracy: 0.7188
Epoch 5/300
18/18 [==============================] - 0s 881us/step - loss: 0.5451 - accuracy: 0.7076
Epoch 6/300
18/18 [==============================] - 0s 677us/step - loss: 0.5415 - accuracy: 0.7132
Epoch 7/300
18/18 [==============================] - 0s 740us/step - loss: 0.5541 - accuracy: 0.7095
Epoch 8/300
18/18 [==============================] - 0s 765us/step - loss: 0.5694 - accuracy: 0.6946
Epoch 9/300
18/18 [==============================] - 0s 823us/step - loss: 0.5604 - accuracy: 0.6965
Epoch 10/300
18/18 [==============================] - 0s 761us/step - loss: 0.5623 - accura

## Challenge 2:
How can you tune the model to improve the accuracy to be higher than what you have obtained?

In [38]:
import keras.optimizers

model = Sequential()
model.add(Dense(24, input_dim=8, activation='relu'))
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(1, input_dim=8, activation='sigmoid'))

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=1000,
#     decay_rate=0.9)

# optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])



In [40]:
model.fit(x_train, y_train, epochs=300, batch_size=30)
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/300
18/18 [==============================] - 0s 1ms/step - loss: 0.4503 - accuracy: 0.7803
Epoch 2/300
18/18 [==============================] - 0s 1ms/step - loss: 0.4450 - accuracy: 0.7952
Epoch 3/300
18/18 [==============================] - 0s 1ms/step - loss: 0.4405 - accuracy: 0.8045
Epoch 4/300
18/18 [==============================] - 0s 1ms/step - loss: 0.4379 - accuracy: 0.8063
Epoch 5/300
18/18 [==============================] - 0s 1ms/step - loss: 0.4459 - accuracy: 0.7858
Epoch 6/300
18/18 [==============================] - 0s 854us/step - loss: 0.4375 - accuracy: 0.7896
Epoch 7/300
18/18 [==============================] - 0s 823us/step - loss: 0.4398 - accuracy: 0.7952
Epoch 8/300
18/18 [==============================] - 0s 823us/step - loss: 0.4373 - accuracy: 0.7914
Epoch 9/300
18/18 [==============================] - 0s 1ms/step - loss: 0.4809 - accuracy: 0.7654
Epoch 10/300
18/18 [==============================] - 0s 913us/step - loss: 0.4414 - accuracy: 0.7970
E

## Challenge 3:
How can you save the model and use it again for prediction directly without (re-)training?

In [41]:
model.save(r'./saved_model')

INFO:tensorflow:Assets written to: ./saved_model\assets
